In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

##########################

# Generazione del dataset sintetico
X, y = make_classification(
    n_samples=1000, 
    n_features=30, 
    n_informative=7, 
    n_redundant=3, 
    random_state=3251
)

##########################

# SCENARIO A: APPROCCIO ERRATO (leakage)
# Errore: Normalizzazione applicata all'intero dataset prima dello split
# Il modello "sbircia" la distribuzione del test set tramite media e dev. std globali
scaler_leak = StandardScaler()
X_leaked = scaler_leak.fit_transform(X) 

X_train_leak, X_test_leak, y_train_leak, y_test_leak = train_test_split(
    X_leaked, y, test_size=0.2, random_state=3251
)

model_leak = LogisticRegression()
model_leak.fit(X_train_leak, y_train_leak)
y_pred_leak = model_leak.predict(X_test_leak)
score_leak = accuracy_score(y_test_leak, y_pred_leak)

print(f"Accuratezza con leakage (stima distorta): {score_leak:.4f}")

##########################

# SCENARIO B: APPROCCIO CORRETTO (pipeline)
# Split dei dati grezzi. I parametri di scaling sono appresi solo sul train set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2025
)

# Utilizziamo una pipeline di Scikit-Learn
# La pipeline garantisce che durante il 'fit', lo scaler veda solo X_train
# Durante il predict, il modello userà le statistiche apprese dal train per trasformare X_test.
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', LogisticRegression())
])

pipeline.fit(X_train, y_train)
y_pred_correct = pipeline.predict(X_test)
score_correct = accuracy_score(y_test, y_pred_correct)

print(f"Accuratezza corretta (stima corretta e realistica): {score_correct:.4f}")

##########################

# Analisi della distorsione
delta = score_leak - score_correct
print(f"Sovrastima della performance dovuta al leakage: {delta:.4f}")

Accuratezza con leakage (stima distorta): 0.7650
Accuratezza corretta (stima corretta e realistica): 0.7150
Sovrastima della performance dovuta al leakage: 0.0500
